# 1D Reactor

In [ ]:
#load modules
import sys
import numpy as np
import pandas as pd
import cantera as ct

#Initialise chemistry
gas = ct.Solution('./ch41step.xml')
R = rxns = ct.Reaction.listFromFile('./ch41step.xml')
myreaction  = R[0]

iCO2 = gas.species_index('CO2')
iH2O = gas.species_index('H2O')
iO2 = gas.species_index('O2')
iCH4 = gas.species_index('CH4')

transient_data = pd.DataFrame()
constant_data = pd.DataFrame()


#run reactor for T = 1000 to 3000
for T in np.arange(1000,3001,1):
    print("T: " +  str(T))
    gas.TPY = T, ct.one_atm, 'CH4:1,O2:2'
    r = ct.IdealGasConstPressureReactor(gas, energy = 'off')
    sim = ct.ReactorNet([r])
    time = 0.0
    states = ct.SolutionArray(gas, extra=['t'])

    transient_data = transient_data.append({'t [s]': sim.time, 'T [K]':  r.T
                                    ,'Yco2': r.Y[iCO2],'Yh2o': r.Y[iH2O],'Yo2':r.Y[iO2],'Ych4':r.Y[iCH4]},ignore_index=True)
    constant_data = constant_data.append({'T [K]':  r.T, 'P [Pa]': r.thermo.P, 'kf': gas.forward_rate_constants, 
                                          'kb':gas.reverse_rate_constants, 'Ea':myreaction.rate.activation_energy, 
                                          'A': myreaction.rate.pre_exponential_factor,
                                          'b': myreaction.rate.temperature_exponent},ignore_index=True)
    for n in range(50):
        time += 5.e-7
        sim.advance(time)
        states.append(r.thermo.state, t=time*1e3)
        transient_data = transient_data.append({'t [s]': sim.time, 'T [K]':  r.T
                                    ,'Yco2': r.Y[iCO2],'Yh2o': r.Y[iH2O],'Yo2':r.Y[iO2],'Ych4':r.Y[iCH4]},ignore_index=True)

#save data
transient_data.to_csv('transient_data_ch41step.csv') 
constant_data.to_csv('constant_data_ch41step.csv') 


# Check kf

In [ ]:
gas.TPX = 2001.0, ct.one_atm, 'CH4:1,O2:2'

A = 3.47850e+08
b = 0.0 
Ea =8.368e+07 #in cal/mol

kf = A*1001**b *np.exp(-Ea/(ct.gas_constant*1001))
kf